<a href="https://colab.research.google.com/github/donghithanh/Cifar100Classification/blob/main/Cifar100_vgg13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True, num_workers=4)

t = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.CIFAR100(root='./data', train=False,download=True, transform=t)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,shuffle=False, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [ ]:
cfg = {
    'A' : [64,     'M', 128,      'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'B' : [64, 64, 'M', 128, 128, 'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'D' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'E' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

In [ ]:
class VGG(nn.Module):

    def __init__(self, features, num_class=100):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, x):
        output = self.features(x)
        output = output.view(output.size()[0], -1)
        output = self.classifier(output)

        return output

def make_layers(cfg, batch_norm=False):
    layers = []

    input_channel = 3
    for l in cfg:
        if l == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers += [nn.Conv2d(input_channel, l, kernel_size=3, padding=1)]

        if batch_norm:
            layers += [nn.BatchNorm2d(l)]

        layers += [nn.ReLU(inplace=True)]
        input_channel = l

    return nn.Sequential(*layers)

In [ ]:
model = VGG(make_layers(cfg['B'], batch_norm=True))
model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

num_epochs = 70
logs = []
for epoch in range(num_epochs):
    total_right = 0
    total = 0
    
    for data in trainloader:
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(),Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        
        predicted = outputs.data.max(1)[1]
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()
        
    print('Epoch {}, Training loss {}, Val accuracy {}'.format(epoch+1,loss,total_right/total))
    logs.append('Epoch {}, Training loss: {}, Val accuracy: {}'.format(epoch, loss, total_right/total))
    if (epoch+1)%5==0:
        torch.save(model,'prt_vgg13_cifar100.ckpt')
    total_right = 0
    total = 0

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1, Training loss 4.178382873535156, Val accuracy 0.020819999277591705
Epoch 2, Training loss 4.0144267082214355, Val accuracy 0.044599998742341995
Epoch 3, Training loss 3.7010509967803955, Val accuracy 0.07131999731063843
Epoch 4, Training loss 3.697787046432495, Val accuracy 0.090379998087883
Epoch 5, Training loss 3.5798192024230957, Val accuracy 0.11499999463558197
Epoch 6, Training loss 3.097090005874634, Val accuracy 0.15014000236988068
Epoch 7, Training loss 2.9505183696746826, Val accuracy 0.19147999584674835
Epoch 8, Training loss 3.1520915031433105, Val accuracy 0.2315399944782257
Epoch 9, Training loss 2.6619691848754883, Val accuracy 0.2668199837207794
Epoch 10, Training loss 2.551929235458374, Val accuracy 0.3055799901485443
Epoch 11, Training loss 2.4772229194641113, Val accuracy 0.34095999598503113
Epoch 12, Training loss 2.4434046745300293, Val accuracy 0.3714599907398224
Epoch 13, Training loss 2.0581650733947754, Val accuracy 0.3982599973678589
Epoch 14, Trainin

In [ ]:
with open('logs_vgg13.txt', 'w') as f:
    for line in logs:
        f.write(line)
        f.write('\n')

In [ ]:
my_model = torch.load('prt_vgg13_cifar100.ckpt')

total_right = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images,labels = data
        images, labels = Variable(images).cuda(),Variable(labels).cuda()
        outputs = my_model(images)

        predicted = outputs.data.max(1)[1]
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()

print("Test accuracy: %d" % (100*total_right/total))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test accuracy: 62


In [ ]:
from torchsummary import summary

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG(make_layers(cfg['B'], batch_norm=True)).to(device)
summary(model,input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,856
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12